# Module 6: LLM 實務應用

## 學習目標
- 理解大型語言模型（LLM）的實務考量
- 學會使用量化（Quantization）載入大型模型
- 掌握 LoRA/PEFT 高效微調技術
- 監控和優化 VRAM 使用

## 為什麼需要這些技術？

現代 LLM 動輒數十億參數：
- **7B 模型**：全精度需要 ~28GB VRAM（fp32）或 ~14GB（fp16）
- **13B 模型**：全精度需要 ~52GB VRAM
- **70B 模型**：需要多張 A100 80GB

你的 RTX 5080 有 16GB VRAM，透過以下技術可以：
1. **量化**：將 7B 模型壓縮到 4-8GB
2. **LoRA**：只訓練極少量參數，節省 VRAM
3. **梯度累積**：模擬大 batch size

---
## Part 1: 環境準備與 VRAM 監控

首先，我們需要安裝一些額外的套件。

In [ ]:
# 安裝必要套件（只需執行一次）
# !pip install transformers accelerate bitsandbytes peft datasets -q

In [ ]:
import torch
import gc
from typing import Optional

def get_gpu_memory_info():
    """獲取詳細的 GPU 記憶體資訊"""
    if not torch.cuda.is_available():
        return "CUDA 不可用"
    
    # 獲取記憶體資訊（以 MB 為單位）
    allocated = torch.cuda.memory_allocated() / 1024**2
    reserved = torch.cuda.memory_reserved() / 1024**2
    total = torch.cuda.get_device_properties(0).total_memory / 1024**2
    free = total - reserved
    
    info = f"""
    GPU: {torch.cuda.get_device_name(0)}
    ────────────────────────────
    已分配 (Allocated): {allocated:,.0f} MB
    已保留 (Reserved):  {reserved:,.0f} MB
    可用 (Free):        {free:,.0f} MB
    總計 (Total):       {total:,.0f} MB
    使用率:             {(reserved/total)*100:.1f}%
    """
    return info

def clear_gpu_memory():
    """清理 GPU 記憶體"""
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    print("GPU 記憶體已清理")

# 顯示初始狀態
print(get_gpu_memory_info())

### VRAM 使用追蹤器

建立一個 context manager 來追蹤操作的 VRAM 消耗：

In [ ]:
class VRAMTracker:
    """追蹤程式碼區塊的 VRAM 使用量"""
    
    def __init__(self, name: str = "Operation"):
        self.name = name
        self.start_allocated = 0
        self.start_reserved = 0
        
    def __enter__(self):
        torch.cuda.synchronize()
        self.start_allocated = torch.cuda.memory_allocated()
        self.start_reserved = torch.cuda.memory_reserved()
        return self
    
    def __exit__(self, *args):
        torch.cuda.synchronize()
        end_allocated = torch.cuda.memory_allocated()
        end_reserved = torch.cuda.memory_reserved()
        
        delta_allocated = (end_allocated - self.start_allocated) / 1024**2
        delta_reserved = (end_reserved - self.start_reserved) / 1024**2
        
        print(f"\n[{self.name}] VRAM 變化:")
        print(f"  分配變化: {delta_allocated:+,.1f} MB")
        print(f"  保留變化: {delta_reserved:+,.1f} MB")

# 測試追蹤器
with VRAMTracker("建立測試張量"):
    # 建立一個 1GB 的張量
    test_tensor = torch.randn(256, 1024, 1024, device='cuda')

# 清理
del test_tensor
clear_gpu_memory()

---
## Part 2: 模型量化 (Quantization)

### 量化的直觀理解

量化就像是「壓縮圖片」：
- **FP32**：每個參數用 32 位元儲存（最精確）
- **FP16/BF16**：每個參數用 16 位元（精度稍降，速度更快）
- **INT8**：每個參數用 8 位元（記憶體減半）
- **INT4**：每個參數用 4 位元（記憶體再減半）

```
7B 模型記憶體需求：
┌─────────────┬────────────┐
│ 精度        │ VRAM 需求  │
├─────────────┼────────────┤
│ FP32        │ ~28 GB     │
│ FP16/BF16   │ ~14 GB     │
│ INT8        │ ~7 GB      │
│ INT4        │ ~3.5 GB    │
└─────────────┴────────────┘
```

### 使用 bitsandbytes 進行 4-bit 量化

bitsandbytes 是目前最流行的量化工具，支援 4-bit 和 8-bit 量化。

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# 4-bit 量化配置
bnb_config_4bit = BitsAndBytesConfig(
    load_in_4bit=True,                      # 使用 4-bit 量化
    bnb_4bit_quant_type="nf4",             # NormalFloat4 量化類型（推薦）
    bnb_4bit_compute_dtype=torch.bfloat16, # 計算時使用 bfloat16
    bnb_4bit_use_double_quant=True,        # 雙重量化，進一步節省記憶體
)

# 8-bit 量化配置
bnb_config_8bit = BitsAndBytesConfig(
    load_in_8bit=True,                      # 使用 8-bit 量化
    llm_int8_threshold=6.0,                # 離群值處理閾值
)

print("量化配置已準備")
print(f"\n4-bit 配置: {bnb_config_4bit.to_dict()}")

### 載入量化模型示例

我們使用一個較小的模型來示範（因為下載 7B 模型需要較長時間）。
在實際應用中，相同的程式碼可以用於任何大小的模型。

In [ ]:
# 使用較小的模型進行示範
# 實際使用時可以換成 "meta-llama/Llama-2-7b-hf" 等更大的模型
MODEL_NAME = "microsoft/DialoGPT-medium"  # 約 355M 參數，適合示範

def load_model_comparison():
    """比較不同精度載入模型的 VRAM 使用"""
    results = {}
    
    # 1. 載入 FP16 模型
    print("\n" + "="*50)
    print("載入 FP16 模型...")
    clear_gpu_memory()
    
    with VRAMTracker("FP16 載入"):
        model_fp16 = AutoModelForCausalLM.from_pretrained(
            MODEL_NAME,
            torch_dtype=torch.float16,
            device_map="cuda"
        )
    results['fp16'] = torch.cuda.memory_allocated() / 1024**2
    
    del model_fp16
    clear_gpu_memory()
    
    # 2. 載入 8-bit 量化模型
    print("\n" + "="*50)
    print("載入 8-bit 量化模型...")
    
    with VRAMTracker("8-bit 載入"):
        model_8bit = AutoModelForCausalLM.from_pretrained(
            MODEL_NAME,
            quantization_config=bnb_config_8bit,
            device_map="cuda"
        )
    results['int8'] = torch.cuda.memory_allocated() / 1024**2
    
    del model_8bit
    clear_gpu_memory()
    
    # 3. 載入 4-bit 量化模型
    print("\n" + "="*50)
    print("載入 4-bit 量化模型...")
    
    with VRAMTracker("4-bit 載入"):
        model_4bit = AutoModelForCausalLM.from_pretrained(
            MODEL_NAME,
            quantization_config=bnb_config_4bit,
            device_map="cuda"
        )
    results['int4'] = torch.cuda.memory_allocated() / 1024**2
    
    del model_4bit
    clear_gpu_memory()
    
    # 顯示比較結果
    print("\n" + "="*50)
    print("VRAM 使用比較：")
    print(f"  FP16:  {results['fp16']:,.0f} MB")
    print(f"  INT8:  {results['int8']:,.0f} MB ({results['int8']/results['fp16']*100:.0f}% of FP16)")
    print(f"  INT4:  {results['int4']:,.0f} MB ({results['int4']/results['fp16']*100:.0f}% of FP16)")
    
    return results

# 執行比較（可能需要幾分鐘下載模型）
# memory_comparison = load_model_comparison()
print("取消註解上面這行來執行 VRAM 比較測試")

### 載入真正的 7B 模型

以下是載入 7B 模型的完整範例（需要 Hugging Face 帳號和模型存取權限）：

In [ ]:
def load_7b_model_4bit(model_name: str = "meta-llama/Llama-2-7b-hf"):
    """
    載入 7B 模型的標準流程
    
    常用的 7B 模型：
    - "meta-llama/Llama-2-7b-hf" (需要申請存取)
    - "mistralai/Mistral-7B-v0.1"
    - "Qwen/Qwen2-7B"
    - "tiiuae/falcon-7b"
    """
    
    print(f"正在載入 {model_name}...")
    print(f"預計 VRAM 使用: ~4-5 GB (4-bit 量化)")
    
    # 量化配置
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
    )
    
    # 載入 tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    # 載入模型
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=quantization_config,
        device_map="auto",  # 自動分配到可用設備
        trust_remote_code=True,  # 某些模型需要
    )
    
    print(f"\n模型載入完成！")
    print(get_gpu_memory_info())
    
    return model, tokenizer

# 範例用法（取消註解執行）：
# model, tokenizer = load_7b_model_4bit("mistralai/Mistral-7B-v0.1")

---
## Part 3: LoRA - 低秩適應 (Low-Rank Adaptation)

### LoRA 的直觀理解

想像你有一本百科全書（預訓練模型），你想讓它學會新的專業知識。

**傳統微調**：重新印刷整本百科全書（修改所有參數）
- 成本高、需要大量資源

**LoRA 微調**：在書頁邊緣加上便利貼（只添加少量參數）
- 原書不變，便利貼記錄修改
- 成本低、可以有多套便利貼（多個任務）

```
原始權重矩陣 W (d × k)：
┌─────────────────────┐
│                     │
│    凍結，不訓練      │  ~7B 參數
│                     │
└─────────────────────┘

LoRA 分解：W' = W + A × B

矩陣 A (d × r)    矩陣 B (r × k)
┌───┐             ┌─────────┐
│   │             │         │
│   │      ×      │         │  只訓練 A 和 B
│   │             └─────────┘  r << d, k
│   │                          例如 r=8
└───┘

參數量比較（以 7B 模型為例）：
- 全量微調: 7,000,000,000 參數
- LoRA (r=8): ~4,000,000 參數 (0.06%)
```

### LoRA 數學原理

對於一個預訓練權重矩陣 $W_0 \in \mathbb{R}^{d \times k}$：

$$W = W_0 + \Delta W = W_0 + BA$$

其中：
- $B \in \mathbb{R}^{d \times r}$
- $A \in \mathbb{R}^{r \times k}$
- $r \ll \min(d, k)$（秩遠小於維度）

前向傳播：
$$h = W_0 x + \frac{\alpha}{r} BA x$$

$\alpha$ 是縮放因子，用於控制 LoRA 的影響程度。

In [ ]:
import torch.nn as nn

class LoRALayer(nn.Module):
    """
    LoRA 層的簡化實現
    
    這個實現展示了 LoRA 的核心概念：
    用兩個小矩陣的乘積來近似權重更新
    """
    
    def __init__(
        self,
        original_layer: nn.Linear,
        r: int = 8,           # LoRA 秩
        alpha: float = 16,    # 縮放因子
        dropout: float = 0.1
    ):
        super().__init__()
        
        self.original_layer = original_layer
        self.r = r
        self.alpha = alpha
        self.scaling = alpha / r
        
        in_features = original_layer.in_features
        out_features = original_layer.out_features
        
        # 凍結原始權重
        for param in self.original_layer.parameters():
            param.requires_grad = False
        
        # LoRA 矩陣 A 和 B
        # A: 用高斯初始化
        # B: 用零初始化（確保訓練開始時 LoRA 沒有影響）
        self.lora_A = nn.Parameter(torch.randn(r, in_features) * 0.01)
        self.lora_B = nn.Parameter(torch.zeros(out_features, r))
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        # 原始輸出
        original_output = self.original_layer(x)
        
        # LoRA 輸出: (B @ A) @ x
        lora_output = self.dropout(x) @ self.lora_A.T @ self.lora_B.T
        
        return original_output + lora_output * self.scaling
    
    def get_trainable_params(self):
        """返回可訓練參數數量"""
        return self.lora_A.numel() + self.lora_B.numel()

# 示範
print("LoRA 層示範：")
print("="*50)

# 模擬一個 Transformer 中的大型線性層
d_model = 4096
original_linear = nn.Linear(d_model, d_model)
original_params = sum(p.numel() for p in original_linear.parameters())

# 包裝成 LoRA 層
lora_layer = LoRALayer(original_linear, r=8, alpha=16)
lora_params = lora_layer.get_trainable_params()

print(f"原始線性層參數: {original_params:,}")
print(f"LoRA 可訓練參數: {lora_params:,}")
print(f"參數比例: {lora_params/original_params*100:.4f}%")

# 測試前向傳播
x = torch.randn(1, 32, d_model)  # [batch, seq_len, d_model]
output = lora_layer(x)
print(f"\n輸入形狀: {x.shape}")
print(f"輸出形狀: {output.shape}")

### 使用 PEFT 庫進行 LoRA 微調

PEFT (Parameter-Efficient Fine-Tuning) 是 Hugging Face 的官方庫，
提供了 LoRA 的標準化實現。

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

def create_lora_config(
    r: int = 8,
    alpha: int = 16,
    dropout: float = 0.05,
    target_modules: list = None
):
    """
    建立 LoRA 配置
    
    參數說明：
    - r: LoRA 秩，越大表達能力越強但參數更多
    - alpha: 縮放因子，通常設為 2*r 或更高
    - dropout: 正則化
    - target_modules: 要應用 LoRA 的層名稱
    
    常見 target_modules：
    - LLaMA: ["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
    - GPT-2: ["c_attn", "c_proj"]
    - BERT: ["query", "value"]
    """
    
    if target_modules is None:
        # 預設針對注意力層
        target_modules = ["q_proj", "v_proj"]
    
    config = LoraConfig(
        r=r,
        lora_alpha=alpha,
        target_modules=target_modules,
        lora_dropout=dropout,
        bias="none",  # 通常不訓練 bias
        task_type=TaskType.CAUSAL_LM,  # 因果語言模型
    )
    
    return config

# 顯示不同配置的參數量估算
print("LoRA 參數量估算 (7B 模型)：")
print("="*50)

# 假設 7B 模型有 32 層，每層 4 個注意力矩陣
d_model = 4096
n_layers = 32
n_matrices_per_layer = 4  # Q, K, V, O

for r in [4, 8, 16, 32, 64]:
    # 每個 LoRA 適配器: 2 * d_model * r 參數
    params_per_matrix = 2 * d_model * r
    total_params = params_per_matrix * n_matrices_per_layer * n_layers
    
    print(f"r={r:2d}: {total_params/1e6:,.1f}M 參數 ({total_params/7e9*100:.3f}% of 7B)")

### 完整的 LoRA 微調流程

In [ ]:
from peft import prepare_model_for_kbit_training
from transformers import TrainingArguments, Trainer
from datasets import Dataset

def setup_lora_training(
    model_name: str,
    lora_r: int = 8,
    lora_alpha: int = 16,
    target_modules: list = None
):
    """
    設置 LoRA 微調的完整流程
    
    這個函數展示了如何：
    1. 載入量化模型
    2. 準備模型進行訓練
    3. 應用 LoRA 配置
    """
    
    print("步驟 1: 載入量化模型...")
    
    # 量化配置
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
    )
    
    # 載入模型
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto",
    )
    
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    print("步驟 2: 準備模型進行 k-bit 訓練...")
    
    # 準備量化模型進行訓練（處理梯度 checkpointing 等）
    model = prepare_model_for_kbit_training(model)
    
    print("步驟 3: 應用 LoRA 配置...")
    
    # LoRA 配置
    if target_modules is None:
        target_modules = ["q_proj", "v_proj"]  # 最小配置
    
    lora_config = LoraConfig(
        r=lora_r,
        lora_alpha=lora_alpha,
        target_modules=target_modules,
        lora_dropout=0.05,
        bias="none",
        task_type=TaskType.CAUSAL_LM,
    )
    
    # 應用 LoRA
    model = get_peft_model(model, lora_config)
    
    # 顯示可訓練參數
    model.print_trainable_parameters()
    
    print(f"\n當前 VRAM 使用:")
    print(get_gpu_memory_info())
    
    return model, tokenizer

# 範例用法（取消註解執行）：
# model, tokenizer = setup_lora_training("microsoft/DialoGPT-medium")

---
## Part 4: 訓練優化技巧

### 梯度累積 (Gradient Accumulation)

當 VRAM 不足以容納大 batch size 時，可以用梯度累積來模擬：

In [ ]:
def gradient_accumulation_demo():
    """
    梯度累積的原理示範
    
    假設你想要 batch_size=32，但 GPU 只能容納 batch_size=4：
    - 設定 accumulation_steps=8
    - 每次處理 4 個樣本，累積 8 次梯度
    - 等效於 batch_size=32
    """
    
    # 創建一個簡單模型
    model = nn.Sequential(
        nn.Linear(100, 50),
        nn.ReLU(),
        nn.Linear(50, 10)
    ).cuda()
    
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.CrossEntropyLoss()
    
    # 參數設定
    effective_batch_size = 32
    micro_batch_size = 4
    accumulation_steps = effective_batch_size // micro_batch_size
    
    print(f"有效 batch size: {effective_batch_size}")
    print(f"微 batch size: {micro_batch_size}")
    print(f"累積步數: {accumulation_steps}")
    print("="*50)
    
    # 模擬一個 epoch
    total_samples = 128
    n_micro_batches = total_samples // micro_batch_size
    
    optimizer.zero_grad()  # 開始時清零梯度
    
    for step in range(n_micro_batches):
        # 生成假資料
        x = torch.randn(micro_batch_size, 100).cuda()
        y = torch.randint(0, 10, (micro_batch_size,)).cuda()
        
        # 前向傳播
        output = model(x)
        loss = criterion(output, y)
        
        # 重要：損失除以累積步數！
        # 這確保了累積後的平均梯度與大 batch 一致
        loss = loss / accumulation_steps
        
        # 反向傳播（梯度會累積）
        loss.backward()
        
        # 每 accumulation_steps 步更新一次參數
        if (step + 1) % accumulation_steps == 0:
            optimizer.step()  # 更新參數
            optimizer.zero_grad()  # 清零梯度
            
            update_step = (step + 1) // accumulation_steps
            print(f"更新步 {update_step}: 累積了 {accumulation_steps} 個微 batch")
    
    print("\n訓練完成！")

gradient_accumulation_demo()

### 混合精度訓練 (Mixed Precision Training)

使用 FP16/BF16 進行計算可以：
1. 減少 VRAM 使用
2. 加速訓練（現代 GPU 對低精度計算有優化）

In [ ]:
from torch.cuda.amp import autocast, GradScaler

def mixed_precision_training_demo():
    """
    混合精度訓練示範
    
    PyTorch 的 autocast 和 GradScaler 讓這變得很簡單：
    - autocast: 自動將適當的操作轉換為 FP16
    - GradScaler: 防止梯度下溢
    """
    
    # 創建模型
    model = nn.Sequential(
        nn.Linear(1024, 512),
        nn.ReLU(),
        nn.Linear(512, 256),
        nn.ReLU(),
        nn.Linear(256, 10)
    ).cuda()
    
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.CrossEntropyLoss()
    scaler = GradScaler()  # 梯度縮放器
    
    # 比較 FP32 vs FP16 的記憶體使用
    print("混合精度訓練示範")
    print("="*50)
    
    # FP32 訓練
    x = torch.randn(64, 1024).cuda()
    y = torch.randint(0, 10, (64,)).cuda()
    
    clear_gpu_memory()
    
    with VRAMTracker("FP32 前向+反向"):
        output = model(x)
        loss = criterion(output, y)
        loss.backward()
    
    optimizer.zero_grad()
    clear_gpu_memory()
    
    # FP16 混合精度訓練
    with VRAMTracker("FP16 混合精度前向+反向"):
        with autocast(dtype=torch.float16):  # 自動混合精度
            output = model(x)
            loss = criterion(output, y)
        
        # 使用 scaler 進行反向傳播
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
    
    print("\n混合精度訓練可以節省顯著的 VRAM！")

mixed_precision_training_demo()

### 梯度檢查點 (Gradient Checkpointing)

犧牲一些計算時間來換取 VRAM：

In [ ]:
from torch.utils.checkpoint import checkpoint

class LargeModel(nn.Module):
    """一個有多層的大模型"""
    
    def __init__(self, n_layers=8, d_model=512):
        super().__init__()
        
        self.layers = nn.ModuleList([
            nn.Sequential(
                nn.Linear(d_model, d_model * 4),
                nn.GELU(),
                nn.Linear(d_model * 4, d_model),
            )
            for _ in range(n_layers)
        ])
        self.output = nn.Linear(d_model, 10)
        
    def forward(self, x, use_checkpointing=False):
        for layer in self.layers:
            if use_checkpointing:
                # 使用 checkpointing：不保存中間激活值
                # 反向傳播時會重新計算
                x = checkpoint(layer, x, use_reentrant=False)
            else:
                x = layer(x)
        return self.output(x)

def checkpointing_comparison():
    """比較有無 checkpointing 的 VRAM 使用"""
    
    print("梯度檢查點比較")
    print("="*50)
    
    # 使用較大的模型和批次來看差異
    model = LargeModel(n_layers=16, d_model=1024).cuda()
    x = torch.randn(32, 128, 1024).cuda()
    
    criterion = nn.CrossEntropyLoss()
    
    # 無 checkpointing
    clear_gpu_memory()
    
    with VRAMTracker("無 Checkpointing"):
        output = model(x, use_checkpointing=False)
        loss = criterion(output.mean(dim=1), torch.zeros(32, dtype=torch.long).cuda())
        loss.backward()
    
    model.zero_grad()
    clear_gpu_memory()
    
    # 有 checkpointing
    with VRAMTracker("有 Checkpointing"):
        output = model(x, use_checkpointing=True)
        loss = criterion(output.mean(dim=1), torch.zeros(32, dtype=torch.long).cuda())
        loss.backward()
    
    print("\nCheckpointing 用計算換記憶體：")
    print("- 訓練速度略慢（約 20-30%）")
    print("- 但 VRAM 使用大幅減少！")

checkpointing_comparison()

---
## Part 5: 完整的 LLM 微調範例

以下是一個完整的、可在 16GB VRAM 上運行的 LLM 微調範例。

In [ ]:
# 完整的 LoRA 微調設定
def get_training_config(output_dir: str = "./lora_output"):
    """
    為 16GB VRAM 優化的訓練配置
    
    這些參數經過調整，可以在 RTX 5080 16GB 上穩定運行
    """
    
    training_args = TrainingArguments(
        output_dir=output_dir,
        
        # Batch size 設定
        per_device_train_batch_size=1,      # 小 batch 適應 VRAM
        gradient_accumulation_steps=16,      # 累積到有效 batch=16
        
        # 訓練步數
        num_train_epochs=1,
        max_steps=100,  # 示範用，實際可能需要更多
        
        # 優化器設定
        learning_rate=2e-4,
        optim="paged_adamw_8bit",  # 8-bit Adam 節省記憶體
        weight_decay=0.01,
        
        # 學習率調度
        warmup_steps=10,
        lr_scheduler_type="cosine",
        
        # 精度設定
        fp16=False,  # RTX 30/40/50 系列建議用 bf16
        bf16=True,
        
        # 記憶體優化
        gradient_checkpointing=True,
        
        # 日誌設定
        logging_steps=10,
        save_steps=50,
        
        # 其他
        report_to="none",  # 不使用 wandb 等
        remove_unused_columns=False,
    )
    
    return training_args

# 顯示配置
config = get_training_config()
print("訓練配置摘要：")
print("="*50)
print(f"有效 batch size: {config.per_device_train_batch_size * config.gradient_accumulation_steps}")
print(f"精度: {'bf16' if config.bf16 else 'fp16' if config.fp16 else 'fp32'}")
print(f"學習率: {config.learning_rate}")
print(f"梯度檢查點: {config.gradient_checkpointing}")
print(f"優化器: {config.optim}")

In [ ]:
def prepare_dataset_for_finetuning(tokenizer, max_length=512):
    """
    準備用於微調的資料集
    
    這裡使用一個簡單的指令資料集格式
    """
    
    # 示範資料（實際應用中會使用真實資料集）
    sample_data = [
        {
            "instruction": "What is machine learning?",
            "response": "Machine learning is a subset of artificial intelligence that enables systems to learn and improve from experience without being explicitly programmed."
        },
        {
            "instruction": "Explain what a neural network is.",
            "response": "A neural network is a computational model inspired by the human brain. It consists of interconnected nodes (neurons) organized in layers that process and transform data to learn patterns."
        },
        {
            "instruction": "What is deep learning?",
            "response": "Deep learning is a type of machine learning that uses neural networks with many layers (deep networks) to learn hierarchical representations of data."
        },
    ]
    
    # 格式化文本
    def format_prompt(example):
        return f"### Instruction:\n{example['instruction']}\n\n### Response:\n{example['response']}"
    
    # 建立資料集
    texts = [format_prompt(ex) for ex in sample_data]
    
    # Tokenize
    def tokenize_function(examples):
        tokenized = tokenizer(
            examples["text"],
            truncation=True,
            max_length=max_length,
            padding="max_length",
            return_tensors="pt"
        )
        # 對於因果語言模型，labels = input_ids
        tokenized["labels"] = tokenized["input_ids"].clone()
        return tokenized
    
    dataset = Dataset.from_dict({"text": texts})
    tokenized_dataset = dataset.map(
        tokenize_function,
        batched=True,
        remove_columns=["text"]
    )
    
    print(f"資料集大小: {len(tokenized_dataset)} 筆")
    print(f"範例文本:\n{texts[0][:200]}...")
    
    return tokenized_dataset

print("資料集準備函數已定義")

In [ ]:
def run_complete_finetuning(
    model_name: str = "microsoft/DialoGPT-medium",
    output_dir: str = "./lora_finetuned",
    max_steps: int = 50
):
    """
    執行完整的 LoRA 微調流程
    
    這個函數整合了所有步驟：
    1. 載入量化模型
    2. 設置 LoRA
    3. 準備資料
    4. 訓練
    5. 保存
    """
    
    print("="*60)
    print("開始 LoRA 微調")
    print("="*60)
    
    # 步驟 1: 載入模型
    print("\n[1/5] 載入量化模型...")
    
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
    )
    
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto",
    )
    
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    print(f"模型載入完成")
    print(get_gpu_memory_info())
    
    # 步驟 2: 準備 LoRA
    print("\n[2/5] 設置 LoRA...")
    
    model = prepare_model_for_kbit_training(model)
    
    lora_config = LoraConfig(
        r=8,
        lora_alpha=16,
        target_modules=["c_attn"],  # GPT-2 風格的注意力層
        lora_dropout=0.05,
        bias="none",
        task_type=TaskType.CAUSAL_LM,
    )
    
    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()
    
    # 步驟 3: 準備資料
    print("\n[3/5] 準備訓練資料...")
    
    train_dataset = prepare_dataset_for_finetuning(tokenizer)
    
    # 步驟 4: 訓練
    print("\n[4/5] 開始訓練...")
    
    training_args = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        num_train_epochs=1,
        max_steps=max_steps,
        learning_rate=2e-4,
        optim="paged_adamw_8bit",
        bf16=True,
        gradient_checkpointing=True,
        logging_steps=10,
        save_steps=max_steps,
        report_to="none",
    )
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
    )
    
    # 訓練
    trainer.train()
    
    print(f"\n訓練完成！")
    print(get_gpu_memory_info())
    
    # 步驟 5: 保存
    print("\n[5/5] 保存 LoRA 權重...")
    
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    
    print(f"\n模型已保存至: {output_dir}")
    print("只保存了 LoRA 權重，檔案非常小！")
    
    return model, tokenizer

# 執行微調（取消註解執行）
# model, tokenizer = run_complete_finetuning(max_steps=30)
print("取消註解上面這行來執行完整的微調流程")

### 載入和使用微調後的模型

In [ ]:
from peft import PeftModel

def load_finetuned_model(
    base_model_name: str,
    lora_path: str
):
    """
    載入微調後的模型進行推理
    
    LoRA 的優點之一：可以快速切換不同的 adapter
    """
    
    print("載入基礎模型...")
    
    # 載入量化的基礎模型
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
    )
    
    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_name,
        quantization_config=bnb_config,
        device_map="auto",
    )
    
    tokenizer = AutoTokenizer.from_pretrained(base_model_name)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    print("載入 LoRA 權重...")
    
    # 載入 LoRA adapter
    model = PeftModel.from_pretrained(base_model, lora_path)
    
    # 可選：合併 LoRA 權重到基礎模型（推理更快，但失去切換能力）
    # model = model.merge_and_unload()
    
    print("模型載入完成！")
    
    return model, tokenizer

def generate_text(model, tokenizer, prompt: str, max_new_tokens: int = 100):
    """
    使用模型生成文本
    """
    
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.pad_token_id,
        )
    
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

print("推理函數已定義")

---
## Part 6: VRAM 優化策略總結

### 16GB VRAM 優化清單

In [ ]:
vram_optimization_guide = """
╔══════════════════════════════════════════════════════════════════════╗
║                    16GB VRAM 優化策略                                ║
╠══════════════════════════════════════════════════════════════════════╣
║                                                                      ║
║  📊 模型載入                                                         ║
║  ├─ 4-bit 量化 (QLoRA)           節省 ~75% VRAM                     ║
║  ├─ 8-bit 量化                   節省 ~50% VRAM                     ║
║  └─ BF16/FP16                    節省 ~50% VRAM                     ║
║                                                                      ║
║  🔧 訓練優化                                                         ║
║  ├─ LoRA (r=8)                   只訓練 ~0.1% 參數                  ║
║  ├─ 梯度累積                     模擬大 batch size                  ║
║  ├─ 梯度檢查點                   用計算換記憶體                     ║
║  └─ 8-bit 優化器                 優化器狀態記憶體減半               ║
║                                                                      ║
║  💡 最佳實踐                                                         ║
║  ├─ batch_size=1 + gradient_accumulation                            ║
║  ├─ 使用 bf16（RTX 30/40/50 系列）                                  ║
║  ├─ 定期清理 GPU 記憶體                                             ║
║  └─ 監控 VRAM 使用，避免 OOM                                        ║
║                                                                      ║
║  📈 可處理的模型大小 (16GB VRAM)                                    ║
║  ├─ 推理: 7B-13B (4-bit 量化)                                       ║
║  ├─ LoRA 微調: 7B (4-bit + LoRA)                                    ║
║  └─ 全量微調: ~1-2B                                                 ║
║                                                                      ║
╚══════════════════════════════════════════════════════════════════════╝
"""

print(vram_optimization_guide)

---
## 練習題

### 練習 1: VRAM 監控工具改進

**目標**: 建立一個可以持續監控 VRAM 並在超過閾值時發出警告的工具

**提示**: 使用 threading 在背景監控

In [ ]:
# 練習 1: VRAM 監控器
# Hint: 使用 threading.Thread 在背景執行監控

import threading
import time

class VRAMMonitor:
    """
    持續監控 VRAM 使用的工具
    
    功能：
    - 在背景持續監控
    - 超過閾值時發出警告
    - 記錄峰值使用量
    """
    
    def __init__(self, warning_threshold_gb: float = 14.0, check_interval: float = 1.0):
        """
        初始化監控器
        
        Args:
            warning_threshold_gb: 警告閾值（GB）
            check_interval: 檢查間隔（秒）
        """
        self.warning_threshold = warning_threshold_gb * 1024  # 轉換為 MB
        self.check_interval = check_interval
        self.running = False
        self.peak_usage = 0
        self.thread = None
        self.history = []
        
    def _monitor_loop(self):
        """背景監控循環"""
        while self.running:
            if torch.cuda.is_available():
                # 獲取當前使用量
                current_usage = torch.cuda.memory_allocated() / 1024**2  # MB
                reserved = torch.cuda.memory_reserved() / 1024**2
                
                # 更新峰值
                if current_usage > self.peak_usage:
                    self.peak_usage = current_usage
                
                # 記錄歷史
                self.history.append({
                    'time': time.time(),
                    'allocated': current_usage,
                    'reserved': reserved
                })
                
                # 檢查警告
                if reserved > self.warning_threshold:
                    print(f"⚠️ VRAM 警告! 使用: {reserved:.0f} MB / 閾值: {self.warning_threshold:.0f} MB")
            
            time.sleep(self.check_interval)
    
    def start(self):
        """開始監控"""
        if self.running:
            print("監控器已在運行")
            return
        
        self.running = True
        self.thread = threading.Thread(target=self._monitor_loop, daemon=True)
        self.thread.start()
        print(f"VRAM 監控已啟動 (警告閾值: {self.warning_threshold/1024:.1f} GB)")
    
    def stop(self):
        """停止監控"""
        self.running = False
        if self.thread:
            self.thread.join(timeout=2)
        print(f"監控已停止。峰值使用: {self.peak_usage:.0f} MB")
    
    def get_stats(self):
        """獲取統計資訊"""
        if not self.history:
            return "無歷史資料"
        
        allocated_values = [h['allocated'] for h in self.history]
        reserved_values = [h['reserved'] for h in self.history]
        
        stats = f"""
        VRAM 統計:
        ─────────────────────
        分配記憶體:
          平均: {sum(allocated_values)/len(allocated_values):.0f} MB
          最大: {max(allocated_values):.0f} MB
          最小: {min(allocated_values):.0f} MB
        
        保留記憶體:
          平均: {sum(reserved_values)/len(reserved_values):.0f} MB
          最大: {max(reserved_values):.0f} MB
          最小: {min(reserved_values):.0f} MB
        
        記錄點數: {len(self.history)}
        """
        return stats

# 測試監控器
monitor = VRAMMonitor(warning_threshold_gb=12.0, check_interval=0.5)
monitor.start()

# 模擬一些 GPU 操作
tensors = []
for i in range(5):
    t = torch.randn(512, 512, 512, device='cuda')
    tensors.append(t)
    time.sleep(0.3)

# 清理
del tensors
clear_gpu_memory()
time.sleep(0.5)

# 停止並查看統計
monitor.stop()
print(monitor.get_stats())

### 練習 2: 自定義 LoRA 層的改進

**目標**: 為 LoRA 層添加 dropout 和可選的權重合併功能

**提示**: 合併時計算 `W + scaling * B @ A`

In [ ]:
# 練習 2: 改進的 LoRA 層

class ImprovedLoRALayer(nn.Module):
    """
    改進的 LoRA 層，增加：
    1. Dropout 正則化
    2. 權重合併功能
    3. 動態秩調整
    """
    
    def __init__(
        self,
        original_layer: nn.Linear,
        r: int = 8,
        alpha: float = 16,
        dropout: float = 0.1
    ):
        super().__init__()
        
        self.original_layer = original_layer
        self.r = r
        self.alpha = alpha
        self.scaling = alpha / r
        self.merged = False  # 追蹤是否已合併
        
        in_features = original_layer.in_features
        out_features = original_layer.out_features
        
        # 凍結原始權重
        for param in self.original_layer.parameters():
            param.requires_grad = False
        
        # LoRA 矩陣
        self.lora_A = nn.Parameter(torch.randn(r, in_features) * 0.01)
        self.lora_B = nn.Parameter(torch.zeros(out_features, r))
        
        # Dropout
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        # 如果已合併，直接使用原始層
        if self.merged:
            return self.original_layer(x)
        
        # 原始輸出
        original_output = self.original_layer(x)
        
        # LoRA 輸出（帶 dropout）
        dropped_x = self.dropout(x)
        lora_output = dropped_x @ self.lora_A.T @ self.lora_B.T
        
        return original_output + lora_output * self.scaling
    
    def merge_weights(self):
        """
        將 LoRA 權重合併到原始層
        合併後推理更快，但無法繼續訓練
        """
        if self.merged:
            print("權重已經合併過了")
            return
        
        # 計算 LoRA 增量: B @ A
        delta_w = self.lora_B @ self.lora_A  # [out_features, in_features]
        
        # 合併到原始權重
        with torch.no_grad():
            self.original_layer.weight.add_(delta_w * self.scaling)
        
        self.merged = True
        print("LoRA 權重已合併到原始層")
    
    def unmerge_weights(self):
        """
        取消合併（恢復原始權重）
        """
        if not self.merged:
            print("權重尚未合併")
            return
        
        # 計算 LoRA 增量
        delta_w = self.lora_B @ self.lora_A
        
        # 從原始權重中減去
        with torch.no_grad():
            self.original_layer.weight.sub_(delta_w * self.scaling)
        
        self.merged = False
        print("LoRA 權重已取消合併")
    
    def get_lora_state_dict(self):
        """
        只獲取 LoRA 權重（用於保存）
        """
        return {
            'lora_A': self.lora_A.data.clone(),
            'lora_B': self.lora_B.data.clone(),
            'r': self.r,
            'alpha': self.alpha,
        }

# 測試改進的 LoRA 層
print("測試改進的 LoRA 層")
print("="*50)

# 創建層
d = 256
original = nn.Linear(d, d)
lora = ImprovedLoRALayer(original, r=8, alpha=16)

# 測試輸入
x = torch.randn(4, 32, d)

# 未合併時的輸出
output_unmerged = lora(x)
print(f"未合併輸出形狀: {output_unmerged.shape}")

# 合併權重
lora.merge_weights()
output_merged = lora(x)
print(f"合併後輸出形狀: {output_merged.shape}")

# 驗證輸出一致性（訓練模式下因 dropout 會略有不同）
lora.eval()
lora.unmerge_weights()
output1 = lora(x)
lora.merge_weights()
output2 = lora(x)

diff = (output1 - output2).abs().max().item()
print(f"\n合併前後輸出差異（eval 模式）: {diff:.10f}")
print("差異應該接近 0（浮點誤差範圍內）")

# 獲取 LoRA 狀態
state = lora.get_lora_state_dict()
print(f"\nLoRA 狀態: r={state['r']}, alpha={state['alpha']}")
print(f"lora_A 形狀: {state['lora_A'].shape}")
print(f"lora_B 形狀: {state['lora_B'].shape}")

### 練習 3: QLoRA 記憶體估算器

**目標**: 建立一個工具來估算不同模型大小和配置下的 VRAM 需求

**提示**: 考慮模型權重、優化器狀態、梯度、激活值等

In [ ]:
# 練習 3: VRAM 需求估算器

class VRAMEstimator:
    """
    估算不同配置下的 VRAM 需求
    
    考慮因素：
    1. 模型權重
    2. 優化器狀態
    3. 梯度
    4. 激活值（前向傳播）
    5. 臨時緩衝區
    """
    
    # 不同精度的每參數位元數
    BITS_PER_PARAM = {
        'fp32': 32,
        'fp16': 16,
        'bf16': 16,
        'int8': 8,
        'int4': 4,
    }
    
    # 優化器狀態乘數（相對於模型大小）
    OPTIMIZER_OVERHEAD = {
        'sgd': 0,           # 無狀態
        'adam': 2,          # m 和 v
        'adamw': 2,
        'adam_8bit': 1,     # 8-bit 量化狀態
        'paged_adamw_8bit': 1,
    }
    
    @classmethod
    def estimate_model_memory(
        cls,
        num_params_billions: float,
        precision: str = 'fp16'
    ) -> float:
        """
        估算模型權重的記憶體需求（GB）
        """
        bits = cls.BITS_PER_PARAM.get(precision, 16)
        bytes_per_param = bits / 8
        memory_gb = num_params_billions * 1e9 * bytes_per_param / 1024**3
        return memory_gb
    
    @classmethod
    def estimate_training_memory(
        cls,
        num_params_billions: float,
        precision: str = 'bf16',
        optimizer: str = 'adamw',
        batch_size: int = 1,
        seq_length: int = 512,
        use_gradient_checkpointing: bool = True,
        lora_r: int = None,
        num_layers: int = 32,
    ) -> dict:
        """
        估算訓練時的總 VRAM 需求
        
        Returns:
            dict: 包含各部分記憶體使用的詳細資訊
        """
        
        # 1. 模型權重
        model_memory = cls.estimate_model_memory(num_params_billions, precision)
        
        # 2. 確定可訓練參數比例
        if lora_r is not None:
            # LoRA: 只有很小比例的參數可訓練
            # 假設每層有 4 個矩陣需要 LoRA
            d_model = int((num_params_billions * 1e9 / (12 * num_layers)) ** 0.5)
            lora_params = 2 * d_model * lora_r * 4 * num_layers
            trainable_ratio = lora_params / (num_params_billions * 1e9)
        else:
            trainable_ratio = 1.0
        
        # 3. 梯度（只針對可訓練參數）
        gradient_memory = model_memory * trainable_ratio
        
        # 4. 優化器狀態
        optimizer_multiplier = cls.OPTIMIZER_OVERHEAD.get(optimizer, 2)
        optimizer_memory = gradient_memory * optimizer_multiplier
        
        # 5. 激活值（與 batch size 和 seq length 相關）
        # 使用啟發式估算
        if use_gradient_checkpointing:
            # Checkpointing 大幅減少激活值記憶體
            activation_factor = 0.3
        else:
            activation_factor = 1.0
        
        # 激活值估算：約為模型大小的 batch_size * seq_length / 1000 倍
        activation_memory = model_memory * batch_size * seq_length / 2000 * activation_factor
        
        # 6. 臨時緩衝區和碎片化開銷（約 10-20%）
        overhead = 0.15
        
        # 總計
        subtotal = model_memory + gradient_memory + optimizer_memory + activation_memory
        total = subtotal * (1 + overhead)
        
        return {
            'model_weights': model_memory,
            'gradients': gradient_memory,
            'optimizer_states': optimizer_memory,
            'activations': activation_memory,
            'overhead': subtotal * overhead,
            'total': total,
            'trainable_params_ratio': trainable_ratio,
        }
    
    @classmethod
    def print_comparison(
        cls,
        num_params_billions: float = 7,
        batch_size: int = 1,
        seq_length: int = 512,
    ):
        """
        比較不同配置的記憶體需求
        """
        
        print(f"\n{'='*70}")
        print(f"VRAM 需求估算: {num_params_billions}B 模型")
        print(f"Batch size: {batch_size}, Sequence length: {seq_length}")
        print(f"{'='*70}")
        
        configs = [
            ('全量微調 FP16', {'precision': 'fp16', 'optimizer': 'adamw', 'lora_r': None}),
            ('全量微調 BF16 + Checkpointing', {'precision': 'bf16', 'optimizer': 'adamw', 'lora_r': None, 'use_gradient_checkpointing': True}),
            ('8-bit 量化 + LoRA r=8', {'precision': 'int8', 'optimizer': 'paged_adamw_8bit', 'lora_r': 8}),
            ('4-bit 量化 + LoRA r=8 (QLoRA)', {'precision': 'int4', 'optimizer': 'paged_adamw_8bit', 'lora_r': 8}),
            ('4-bit 量化 + LoRA r=16', {'precision': 'int4', 'optimizer': 'paged_adamw_8bit', 'lora_r': 16}),
        ]
        
        for name, config in configs:
            result = cls.estimate_training_memory(
                num_params_billions=num_params_billions,
                batch_size=batch_size,
                seq_length=seq_length,
                **config
            )
            
            print(f"\n{name}:")
            print(f"  模型權重:   {result['model_weights']:6.1f} GB")
            print(f"  梯度:       {result['gradients']:6.1f} GB")
            print(f"  優化器:     {result['optimizer_states']:6.1f} GB")
            print(f"  激活值:     {result['activations']:6.1f} GB")
            print(f"  開銷:       {result['overhead']:6.1f} GB")
            print(f"  ────────────────────")
            print(f"  總計:       {result['total']:6.1f} GB")
            
            if config.get('lora_r'):
                print(f"  可訓練比例: {result['trainable_params_ratio']*100:.4f}%")
            
            # 16GB VRAM 可行性
            if result['total'] <= 16:
                print(f"  ✅ 適合 16GB VRAM")
            else:
                print(f"  ❌ 需要 > 16GB VRAM")

# 執行比較
VRAMEstimator.print_comparison(num_params_billions=7, batch_size=1, seq_length=512)

# 也可以看看更大的模型
print("\n" + "="*70)
print("不同模型大小的 4-bit QLoRA 記憶體需求:")
print("="*70)

for size in [3, 7, 13, 34, 70]:
    result = VRAMEstimator.estimate_training_memory(
        num_params_billions=size,
        precision='int4',
        optimizer='paged_adamw_8bit',
        lora_r=8,
        batch_size=1,
        seq_length=512,
    )
    status = "✅" if result['total'] <= 16 else "❌"
    print(f"{size}B 模型: {result['total']:5.1f} GB {status}")

---
## Part 7: 進階 LLM 技巧

### 7.1 推理優化：KV Cache

**問題：** 自回歸生成時，每生成一個 token 都要重新計算所有之前 token 的 K、V

**解決方案：** 緩存已計算的 K、V 值（KV Cache）

---
## 總結

### 本模組重點

1. **VRAM 監控**
   - 使用 `torch.cuda.memory_allocated()` 和 `memory_reserved()`
   - 定期清理：`gc.collect()` + `torch.cuda.empty_cache()`

2. **模型量化**
   - 4-bit NF4 量化節省 ~75% 記憶體
   - 使用 bitsandbytes 的 `BitsAndBytesConfig`
   - 雙重量化進一步節省空間

3. **LoRA 微調**
   - 只訓練極少量參數（~0.1%）
   - 使用 PEFT 庫的 `LoraConfig` 和 `get_peft_model`
   - 秩 r 越大表達能力越強

4. **訓練優化**
   - 梯度累積模擬大 batch size
   - 混合精度（BF16）加速訓練
   - 梯度檢查點用計算換記憶體
   - 8-bit 優化器減少狀態記憶體

### 16GB VRAM 的實用配置

```python
# 推薦的 7B 模型 QLoRA 配置
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
)

training_args = TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    bf16=True,
    gradient_checkpointing=True,
    optim="paged_adamw_8bit",
)
```

### 下一步

在下一個模組中，我們將學習 **Autoencoder 和 VAE**，探索如何學習資料的潛在表示並生成新樣本。

In [ ]:
# KV Cache 概念說明

print("""
KV Cache 原理：

自回歸生成：每次只生成一個 token
  Step 1: "Hello" → 計算 K₁, V₁ → 生成 "world"
  Step 2: "Hello world" → 重新計算 K₁, V₁, K₂, V₂ → 生成 "!"  ← 浪費！
  Step 3: "Hello world!" → 重新計算 K₁, V₁, K₂, V₂, K₃, V₃ → ...

使用 KV Cache：
  Step 1: "Hello" → 計算並緩存 K₁, V₁ → 生成 "world"
  Step 2: 只計算 K₂, V₂（K₁, V₁ 從緩存讀取）→ 生成 "!"
  Step 3: 只計算 K₃, V₃ → ...

效果：
  - 生成速度提升數倍
  - 代價：需要額外的記憶體存儲 KV Cache

KV Cache 記憶體估算：
  memory = 2 × batch_size × num_layers × seq_len × num_heads × head_dim × bytes_per_element
  
  7B 模型 (假設 32 層, 32 heads, head_dim=128):
    FP16: 2 × 1 × 32 × 2048 × 32 × 128 × 2 bytes = ~1 GB per 2K tokens
""")

# 使用 Transformers 時，KV Cache 是自動處理的
# 只需要在生成時設置 use_cache=True（預設就是 True）

def generate_with_kv_cache_demo():
    """展示 KV Cache 的使用"""
    print("\n使用 HuggingFace 時，KV Cache 自動啟用：")
    print("-" * 50)
    
    # 生成配置示例
    generation_config = {
        "max_new_tokens": 100,
        "do_sample": True,
        "temperature": 0.7,
        "use_cache": True,  # 預設就是 True，啟用 KV Cache
        # "past_key_values": None,  # 可以傳入之前的 KV Cache 繼續生成
    }
    
    print("generation_config:")
    for k, v in generation_config.items():
        print(f"  {k}: {v}")
    
    print("\n進階用法：手動管理 KV Cache")
    print("  outputs = model(input_ids, past_key_values=past_kv, use_cache=True)")
    print("  past_kv = outputs.past_key_values  # 保存用於下一次生成")

generate_with_kv_cache_demo()

### 7.2 Prompt Engineering 基礎

有效的 Prompt 設計對 LLM 輸出品質至關重要：

In [ ]:
# Prompt Engineering 技巧

print("""
╔══════════════════════════════════════════════════════════════════════╗
║                      Prompt Engineering 技巧                         ║
╠══════════════════════════════════════════════════════════════════════╣
║                                                                      ║
║  1. 指令格式 (Instruction Format)                                    ║
║  ────────────────────────────────                                    ║
║  ### Instruction:                                                    ║
║  {任務描述}                                                          ║
║                                                                      ║
║  ### Input:                                                          ║
║  {輸入資料}                                                          ║
║                                                                      ║
║  ### Response:                                                       ║
║                                                                      ║
║  2. Few-shot Learning                                                ║
║  ────────────────────                                                ║
║  提供幾個範例讓模型學習格式：                                         ║
║  Q: What is 2+2?                                                     ║
║  A: 4                                                                ║
║                                                                      ║
║  Q: What is 3+5?                                                     ║
║  A: 8                                                                ║
║                                                                      ║
║  Q: What is 7+9?                                                     ║
║  A:                                                                  ║
║                                                                      ║
║  3. Chain-of-Thought (CoT)                                           ║
║  ─────────────────────────                                           ║
║  讓模型展示推理過程：                                                 ║
║  "Let's think step by step..."                                       ║
║                                                                      ║
║  4. System Prompt (對話模型)                                         ║
║  ─────────────────────────                                           ║
║  設定模型的角色和行為：                                               ║
║  "You are a helpful assistant that..."                               ║
║                                                                      ║
╚══════════════════════════════════════════════════════════════════════╝
""")

# 不同任務的 Prompt 模板
prompt_templates = {
    "classification": '''Classify the following text into one of the categories: {categories}

Text: {text}

Category:''',

    "summarization": '''Summarize the following text in {num_sentences} sentences:

Text: {text}

Summary:''',

    "translation": '''Translate the following text from {source_lang} to {target_lang}:

{text}

Translation:''',

    "code_generation": '''Write a Python function that {task_description}.

Requirements:
{requirements}

```python''',

    "qa": '''Answer the question based on the context below.

Context: {context}

Question: {question}

Answer:''',
}

print("\nPrompt 模板範例：")
print("=" * 50)
for name, template in prompt_templates.items():
    print(f"\n[{name}]")
    print(template[:100] + "...")

---

## Module 6 完整總結

### 🎯 核心技術

| 技術 | 用途 | VRAM 節省 |
|------|------|-----------|
| **4-bit 量化** | 載入大模型 | ~75% |
| **8-bit 量化** | 載入大模型 | ~50% |
| **LoRA** | 高效微調 | 只訓練 ~0.1% 參數 |
| **QLoRA** | 量化 + LoRA | 結合兩者優勢 |
| **梯度累積** | 模擬大 batch | 不直接省 VRAM |
| **梯度檢查點** | 訓練大模型 | ~50% 激活值 |

### 🔑 關鍵配置

```python
# 4-bit 量化（推薦配置）
BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

# LoRA 配置
LoraConfig(
    r=8,                          # 秩（8-64）
    lora_alpha=16,                # 縮放（通常 2*r）
    target_modules=["q_proj", "v_proj"],  # 目標層
    lora_dropout=0.05,
)

# 訓練配置（16GB VRAM）
TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    bf16=True,
    gradient_checkpointing=True,
    optim="paged_adamw_8bit",
)
```

### 📊 16GB VRAM 能力範圍

| 任務 | 可用模型大小 |
|------|-------------|
| **推理** | 7B-13B (4-bit) |
| **QLoRA 微調** | 7B (4-bit + LoRA) |
| **全量微調** | ~1-2B |

### 💡 實務建議

1. **模型選擇**：
   - 7B 模型是 16GB VRAM 的甜蜜點
   - 優先考慮 Mistral、LLaMA 2、Qwen2

2. **微調策略**：
   - 先嘗試 r=8，效果不好再增加
   - target_modules 越多效果越好但 VRAM 越高
   - 從 q_proj, v_proj 開始

3. **推理優化**：
   - 使用 KV Cache（預設啟用）
   - 長文本考慮 vLLM 或 TGI
   - 批次推理可提高吞吐量

4. **常見問題**：
   - OOM → 減少 batch_size 或啟用 gradient_checkpointing
   - 訓練不穩定 → 降低學習率或增加 warmup
   - 效果不好 → 增加 r 或 target_modules

### 🚀 下一步

- Module 7：Autoencoder 與 VAE（學習潛在表示）
- 進一步學習：vLLM、TGI 等推理框架

---

**恭喜完成 LLM 實務模組！** 🎉

你現在已經掌握了在消費級 GPU 上使用和微調大型語言模型的核心技術。QLoRA 是目前最實用的微調方法，讓 7B 模型的微調變得觸手可及。